# Cost of Plutocracy

This article aims to analyze some of the biggest DAOs in the crypto space and see how many of them are influenced by token holders which own a disproportionate amount of tokens compared to most voters.

In [1]:
# sets up the pynb environment
import os
import sys

import pandas as pd
import numpy as np


module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from libs.data_processing.statistics import (
    get_number_of_whales_to_all_voters_ratio,
    get_score_comparisons,
)


The data used for this article came from data sources that specialize in providing data on Decentralised Autonomous Organization (DAO) governance, such as DeepDAO and Snapshot. Coingecko was also used to find market data for DAO native tokens.

Using this data, two spreadsheets were compiled which act as the local database for this analysis. Each spreadsheet contains each voter's choice and voting power for the last fifty proposals each DAO opened. **One spreadsheet filters out "whales"** which, in the context of this analysis, are voters that have voting power **at or above the 95th percentile of voting power for that proposal**.

In [2]:
plutocracy_report_data = pd.read_excel(
    "../plutocracy_report.xlsx", sheet_name=None, engine="openpyxl"
)
plutocracy_report_data_filtered = pd.read_excel(
    "../plutocracy_report_filtered.xlsx", sheet_name=None, engine="openpyxl"
)

With this, we can begin asking ourselves some questions. Such as: **How many whales have voted in the last fifty proposals for each of the DAOs we analyzed?** Voting power for each of these DAOs varies by the type of [voting strategy](https://docs.snapshot.org/strategies/what-is-a-strategy) outlined in their respective Snapshot spaces.

However, to keep this analysis simple, we assume the common `erc-20-balance` strategy and define the cost of voting power (`vp`) as the US dollar value of a DAO's native token.

In [3]:
voting_ratios = get_number_of_whales_to_all_voters_ratio(
    plutocracy_report_data, plutocracy_report_data_filtered
)


In [4]:
pd.DataFrame(
    [list(result.items())[0][1] for result in voting_ratios],
    index=[list(result.items())[0][0] for result in voting_ratios],
    columns=[
        "# of whales",
        "all voters",
        "avg vp for non-whales",
        "avg vp for whales",
        "avg cost of vote",
    ],
)


,# of whales,all voters,avg vp for non-whales,avg vp for whales,avg cost of vote
Uniswap,801,21301,1.658005,9.212467e+04,7.285653
ENS,321,9742,281.564683,5.561220e+04,29.874161
Lido,164,4438,115.172600,1.363868e+06,1.573563
Frax,2,165,110023.913464,1.444174e+07,8.497316
Curve Finance,14,235,3454.929793,1.352748e+06,3.338636
Decentraland,56,1212,238.465901,2.900371e+05,0.706981
Radicle,18,395,26779.601995,9.430134e+05,12.567908
Gitcoin,86,3075,18.195005,2.643118e+05,3.112334


Here we can see that for the DAOs which we choose to analyze the top 5% of voters have a clear, disproportionate amount of voting power compared to the average voter.

We can also gain some insight into the economic might that these whales hold for each DAO at the time of voting by taking a look at the average cost of each vote for each DAO's native token at the time these proposals were active (in USD).

---

We move on to take a look at what DAO participation would look like without whale involvement. We do so by comparing the scores of each proposal and checking whether or not the outcome of the proposal was changed after filtering out whales.

We check whether or not a proposal's outcome has changed simply by checking if the largest vote choice score has changed after filtering out whales, more specifically, in python we do:
```python
has_changed_outcome = not unfiltered_winning_choice_index == filtered_winning_choice_index
```

In [5]:
pd.set_option("display.max_rows", 5)
score_diffrences = get_score_comparisons(
    plutocracy_report_data, plutocracy_report_data_filtered
)


In [6]:
score_diffrences_dfs = dict()

for score_diffrence in score_diffrences:
    for organization, data in score_diffrence.items():
        data: dict[str, list] = data
        items = data.items()
        score_diffrences_dfs[organization] = pd.DataFrame(
            [score_data for _, score_data in items],
            index=pd.Index(
                ([proposal_id for proposal_id, _ in items]), name="Proposal ID"
            ),
            columns=[
                "type",
                "choices",
                "score_diffrences",
                "total_vp",
                "outcome_changed",
                "outcome"
            ],
        )
        score_diffrences_dfs[organization]["total_vp"] = score_diffrences_dfs[
            organization
        ]["total_vp"].apply("{:.9f}".format)
        score_diffrences_dfs[organization].index = score_diffrences_dfs[organization].index.str.slice(0, 9)


First, let's look at Uniswap

In [7]:
score_diffrences_dfs["Uniswap"].index = score_diffrences_dfs["Uniswap"].index.str.slice(0, 9)
score_diffrences_dfs["Uniswap"]

,type,choices,score_diffrences,total_vp,outcome_changed,outcome
Proposal ID,,,,,,
0x374d7f9,single-choice,"['Yes', 'No']","[17079.258062488916, 341.33411472548084]",32011.083070674,False,Yes
0x15031b6,single-choice,"['For', 'Against', 'Abstain']","[15561440.496482328, 0.0, 0.0]",38105686.407799788,False,For
...,...,...,...,...,...,...
QmcpJ8Qm9,single-choice,"['Yes, Bring V3 to AVAX', 'No, V3 Should Stay ...","[9790.514303638305, 0]",10030.326504541,False,"Yes, Bring V3 to AVAX"
QmYeNLn1x,single-choice,"['No support', '25k - 50k (37.5k $UNI loan)', ...","[0.0, 0.0, 75.83513107330648, 135.62208, 0.0, ...",212333.479358470,False,800k+ (1M $UNI loan)


We can do some quick calculations using [pandas](https://pandas.pydata.org/) to see that 12.5% of proposals had their outcome changed by removing whales.

In [8]:
score_diffrences_dfs["Uniswap"]["outcome_changed"].value_counts(normalize=True)

False    0.875
True     0.125
Name: outcome_changed, dtype: float64

One of the proposals which had their outcome changed after removing whales was the proposal to [add a Liquidity Mining Manager for the Optimism-Uniswap Liquidity Mining Program](https://snapshot.org/#/uniswap/proposal/0xfd3d3807bd2a6eda1327c311b83de235061d39ff1bdfb616c9f9b0d367c3ac2c).

In this case, "Overnight Finance" would've been the chosen LMM if it weren't for whale intervention, instead "DeFiEdge" was chosen for this campaign. From the data, we can see that without whale intervention, DeFiEdge would've had ~2,708,025 fewer votes out of the total ~2,728,492 voting power on this proposal. That is ~99.3% of voting power allocated for this proposal directed to choosing "DeFiEdge".

In [9]:
propsal_choices = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0xfd3d3807bd2a6eda1327c311b83de235061d39ff1bdfb616c9f9b0d367c3ac2c'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Uniswap"].loc["0xfd3d380"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0xfd3d3807bd2a6eda1327c311b83de235061d39ff1bdfb616c9f9b0d367c3ac2c'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,add Overnight Finance,add DeFiEdge,add none
Scores,14335.930887,2.710600e+06,3556.089173
Score Differeces,10054.585076,2.708025e+06,3412.851497


One highly contentious proposal whose outcome did not change after filtering out whales was [this temperature check to choose which Eth <> BNB bridge to use for Uniswap v3 governance](https://snapshot.org/#/uniswap/proposal/0x6b8df360fdf73085b21fdf5eef9f85916fbde95621a3d454cb20fbe545ffc852). We see that even for the least popular choices, whales still contributed to the majority of the votes. We also observe that whales held the overwhelming majority of voting power for the runner-up, "LayerZero", with 99.9% of voting power allocated to this proposal being directed to this choice. By comparison, "Wormhole" received 65.1% of voting power for this proposal coming from whales.

In [10]:
propsal_choices = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0x6b8df360fdf73085b21fdf5eef9f85916fbde95621a3d454cb20fbe545ffc852'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Uniswap"].loc["0x6b8df36"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0x6b8df360fdf73085b21fdf5eef9f85916fbde95621a3d454cb20fbe545ffc852'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,Wormhole,LayerZero,Celer,deBridge
Scores,2.840391e+07,1.717334e+07,662.606204,3772.802474
Score Differeces,2.327546e+07,1.716363e+07,481.545728,3402.436704


Next up is ENS.

In [47]:
score_diffrences_dfs["ENS"]

,type,choices,score_diffrences,total_vp,outcome_changed,outcome
Proposal ID,,,,,,
0x41b3509,single-choice,"['For', 'Against', 'Abstain']","[1667210.963067633, 0.0, 137.12454090065714]",1670185.073618348,False,For
0xd7eff78,single-choice,"['For', 'Against', 'Abstain']","[1666938.9315321625, 100.38682608595627, 141.9...",1670154.219044510,False,For
...,...,...,...,...,...,...
0xfe73d1b,approval,"['1. Transfer Treasury', '2. Transfer Registra...","[4573759.0782857565, 4573759.078285757, 457375...",4856109.997858677,False,1. Transfer Treasury
0xd810c4c,approval,"['Ratify Article I', 'Ratify Article II', 'Rat...","[170029.0968926698, 170029.0968926698, 170029....",16102806.718553530,False,Ratify Article I


In [12]:
score_diffrences_dfs["ENS"]["outcome_changed"].value_counts(normalize=True)

False    0.964286
True     0.035714
Name: outcome_changed, dtype: float64

We see that a very small percentage of proposal outcomes were changed after filtering out whales (3.5%). However, if we take look at score differences as they compare to each score, we see that they were almost all heavily backed by whales.

In [13]:
propsal_choices = plutocracy_report_data['ENS'][plutocracy_report_data['ENS']['proposal_id'] == '0x41b3509b88e15677aa15680f48278517f794822fb9a79b9c621def53f1866be7'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["ENS"].loc["0x41b3509"]["score_diffrences"]
proposal_scores = plutocracy_report_data['ENS'][plutocracy_report_data['ENS']['proposal_id'] == '0x41b3509b88e15677aa15680f48278517f794822fb9a79b9c621def53f1866be7'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,For,Against,Abstain
Scores,1.669865e+06,125.702762,194.817417
Score Differeces,1.667211e+06,0.000000,137.124541


[This proposal](https://snapshot.org/#/ens.eth/proposal/0x41b3509b88e15677aa15680f48278517f794822fb9a79b9c621def53f1866be7) once passed will approve the funding of various grants ENS has committed to funding as detailed by the proposal, **the amount of capital to be committed to this proposal is 250k USDC and 50 ETH**. 99.9% of voting power allocated towards this proposal was committed to voting for its passing, 0.0001% was allocated to abstaining and even a smaller percentage was allocated to voting against its passing.

Meaning that ~0.1% of the vote would've still been allocated towards voting for the proposal to pass even without whale intervention, which I think many would see as fine since the funding of public goods is always important.

Another proposal on ENS which had a large amount of voting power committed by whales is [this proposal](https://snapshot.org/#/ens.eth/proposal/0x5788bf0f52ce82a1d3f7750a80f3001671ded49e4e0239dbbafd154275c78f8b) to commit **935k USDC and 254 ETH** to the ENS Ecosystem Working group which is responsible for *growing and improving the ENS Ecosystem by funding people and projects that are ENS-specific or ENS-centric*.

In [14]:
propsal_choices = plutocracy_report_data['ENS'][plutocracy_report_data['ENS']['proposal_id'] == '0x5788bf0f52ce82a1d3f7750a80f3001671ded49e4e0239dbbafd154275c78f8b'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["ENS"].loc["0x5788bf0"]["score_diffrences"]
proposal_scores = plutocracy_report_data['ENS'][plutocracy_report_data['ENS']['proposal_id'] == '0x5788bf0f52ce82a1d3f7750a80f3001671ded49e4e0239dbbafd154275c78f8b'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,For,Against,Abstain
Scores,1.449464e+06,208582.378002,13640.353424
Score Differeces,1.446509e+06,208348.531592,13519.449465


Similar to the previous proposal we looked at, this one was highly influenced by whales (~86.5% of voting power committed to this proposal coming from whales that voted for its passing, ~12.5% from whales that voted against it, and ~0.8% of whales voted to abstain).

Yet, even after removing whales, the proposal still passes. After doing some quick math, we can see the remaining voting power for this proposal that voted for its passing equates to about ~0.17% out of the remaining 0.2% of voting power after removing whales.

Still, I am not at all saying that the outcome of this proposal is controversial. An organisation's funding initiatives that aid in the development of its product should not be a controversial topic for its members. I suppose the only thing that would be contested is the amount that is committed to this proposal.

---
Next let's take a look at everyone's favourite liquid staking platform: Lido.

In [45]:
mask = score_diffrences_dfs["Lido"].apply(
    lambda df: df["outcome_changed"] == True,
    axis=1
)

score_diffrences_dfs["Lido"].loc[mask].astype({"total_vp": "float64"}, copy=False).sort_values("total_vp", ascending=False)

,type,choices,score_diffrences,total_vp,outcome_changed,outcome
Proposal ID,,,,,,
0xede83ac,single-choice,"['For-Keep as listed, no lock up', 'For-Keep a...","[40042.09181, 21165133.189574376, 42456371.986...",6.389112e+07,True,"For-Keep as listed, 1 yr lock up"
0xcbf5343,single-choice,"['1. For 6 new oracles, 6/11 set', '2. For 4 n...","[92934.92264371048, 52723122.2034864, 4535.772...",5.282637e+07,True,"1. For 6 new oracles, 6/11 set"


In [16]:
score_diffrences_dfs["Lido"]["outcome_changed"].value_counts(normalize=True)

False    0.96
True     0.04
Name: outcome_changed, dtype: float64

Here, we see that there was a change in outcome for just two of the proposals (4%). [This proposal](https://snapshot.org/#/lido-snapshot.eth/proposal/0xcbf534335fe07c046caa933e1623ac38bfb3d1890ab825264a0b47415cf7799b) to [expand the oracle set and quorum](https://mainnet.lido.fi/#/lido-dao/0x442af784a788a5bd6f42a01ebe9f287a871243fb/) of Lido DAO oracle node operators was passed 16/12/22 and onboarded 4 new oracle node operators and set the quorum to 5/9 reports from oracles being identical to be accepted by the protocol.

Had whales not intervened, Option 1 to onboard 6 additional oracles and set the quorum to 6/11 would've passed.

In [17]:
propsal_choices = plutocracy_report_data['Lido'][plutocracy_report_data['Lido']['proposal_id'] == '0xcbf534335fe07c046caa933e1623ac38bfb3d1890ab825264a0b47415cf7799b'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Lido"].loc["0xcbf5343"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Lido'][plutocracy_report_data['Lido']['proposal_id'] == '0xcbf534335fe07c046caa933e1623ac38bfb3d1890ab825264a0b47415cf7799b'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,"1. For 6 new oracles, 6/11 set","2. For 4 new oracles, 5/9 set",3. Against
Scores,96886.616711,5.272470e+07,4778.954352
Score Differeces,92934.922644,5.272312e+07,4535.772055


Here we see that whale participation was consistent for all choices for this proposal. In fact, ~99.982% of voting power allocated to this proposal came from whales that voted for each of the outcomes. The second outcome alone had ~99.980% of the voting power allocated to this proposal, so obviously, its removal would make a huge impression on the outcome.

Indeed, the voting power would be comparatively low without whale intervention (if this were the case, only ~9277.7397 LDO would be allocated to this proposal). However the fact that after doing so additional oracles would still be implemented anyway, where the only other alternative would be to not implement any more oracles, shows that the community that participated in this proposal still saw the need to add more oracles to further promote decentralisation of oracles which [report chain state from the beacon chain to the execution layer](https://docs.lido.fi/guides/oracle-operator-manual) of ETH 2.0 as it pertains to the operation of Lido.

---
Now we take a look at Frax.

In [43]:
mask = score_diffrences_dfs["Frax"].apply(
    lambda df: df["outcome_changed"] == True,
    axis=1
)

score_diffrences_dfs["Frax"].loc[mask].astype({"total_vp": "float64"}, copy=False).sort_values("total_vp", ascending=False)

,type,choices,score_diffrences,total_vp,outcome_changed,outcome
Proposal ID,,,,,,
0x9727168,weighted,"['For', 'Against']","[954806.1451496906, 0.0]",1.035478e+06,True,Against


In [19]:
score_diffrences_dfs["Frax"]["outcome_changed"].value_counts(normalize=True)

False    0.98
True     0.02
Name: outcome_changed, dtype: float64

Whilst enormous voting power was filtered out from all the proposals listed, just one had their outcomes changed (representing 2% of all proposals analysed).

If we look at the proposal which attracted the most voting power, [this proposal](https://snapshot.org/#/frax.eth/proposal/0xece8d5be8b180b54350c4bddee190e24e2849d233f8aac11e0ef0aa7d658ae2a) to renew the grant for Flywheel for 2023 (in the amount of approx $214.2k, split 50/50 in FXS and FRAX) for the promotion of "the Frax ecosystem and producing high-quality DeFi content", we see that just over 50% of the voting power was removed from this proposal after filtering out whales (approx 56.7%).

In [20]:
propsal_choices = plutocracy_report_data['Frax'][plutocracy_report_data['Frax']['proposal_id'] == '0xece8d5be8b180b54350c4bddee190e24e2849d233f8aac11e0ef0aa7d658ae2a'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Frax"].loc["0xece8d5b"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Frax'][plutocracy_report_data['Frax']['proposal_id'] == '0xece8d5be8b180b54350c4bddee190e24e2849d233f8aac11e0ef0aa7d658ae2a'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,For,Against
Scores,1.172897e+07,131.88977
Score Differeces,9.238258e+06,0.00000


Though, because the opposition voting power for this proposal was a paltry 1E-5% of the vote, filtering out whales did not change the outcome. This was not a controversial outcome to this proposal after observing the number of voting participants (FXS token holders who participated in this proposal).

---
If we now look at Decentraland, we observe that there is quite a bad case of concentration of votes in the hands of plutocrats.

In [42]:
mask = score_diffrences_dfs["Decentraland"].apply(
    lambda df: df["outcome_changed"] == True,
    axis=1
)

score_diffrences_dfs["Decentraland"].loc[mask].astype({"total_vp": "float64"}, copy=False).sort_values("total_vp", ascending=False)

,type,choices,score_diffrences,total_vp,outcome_changed,outcome
Proposal ID,,,,,,
0x105e115,single-choice,"['AwedJob', 'Champ', 'Martriay', 'Szjanko', 'T...","[133936.60868228367, 177199.1938361379, 777247...",9.925258e+06,True,Szjanko
0xe126f25,single-choice,"['yes', 'no']","[4484370.231829518, 5067699.548291741]",9.704957e+06,True,yes
...,...,...,...,...,...,...
0xdfc9d87,single-choice,"['yes', 'no']","[6498.38425178834, 1122668.0743977013]",1.130732e+06,True,yes
0x36869ff,single-choice,"['yes', 'no']","[473662.4626787397, 260299.90800875027]",7.566330e+05,True,no


In [22]:
score_diffrences_dfs["Decentraland"]["outcome_changed"].value_counts(normalize=True)

False    0.76
True     0.24
Name: outcome_changed, dtype: float64

Here we see that just under a quarter of proposals' outcomes would change if whales were not involved in Decentraland governance.

For example, [this proposal](https://snapshot.org/#/snapshot.dcl.eth/proposal/0x7f6fed8c7645d1b793526564104e4f79864a9e30ae284029f752b6297478b4f5) to set a duration period for the tenure of Decentraland DAO committee members saw 99.9% of voting power concentrated to whales, with 94.85% of proposal voting power being allocated to voting for the proposal to not pass.

In [23]:
propsal_choices = plutocracy_report_data['Decentraland'][plutocracy_report_data['Decentraland']['proposal_id'] == '0x7f6fed8c7645d1b793526564104e4f79864a9e30ae284029f752b6297478b4f5'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Decentraland"].loc["0x7f6fed8"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Decentraland'][plutocracy_report_data['Decentraland']['proposal_id'] == '0x7f6fed8c7645d1b793526564104e4f79864a9e30ae284029f752b6297478b4f5'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,Yes,No,Invalid question/options
Scores,168338.243827,1.394610e+06,0.025206
Score Differeces,166209.638630,1.392896e+06,0.000000


Obviously a case of centralized power voting for their own self interests, its not suprising that if whales were factored out of the proposal (which would've given existing and future committee members set terms, making the roles more democratic) the proposal would've passed, albeit, with very low voting power in comparison.

---
Going back to DeFi, let's now look at Curve Finance.

In [55]:
mask = score_diffrences_dfs["Curve Finance"].apply(
    lambda df: df["outcome_changed"] == True,
    axis=1
)

score_diffrences_dfs["Curve Finance"].loc[mask].astype({"total_vp": "float64"}, copy=False).sort_values("total_vp", ascending=False)

,type,choices,score_diffrences,total_vp,outcome_changed,outcome
Proposal ID,,,,,,
0x0eb23ea,single-choice,"['Yes', 'No']","[23462.994316173717, 511330.4724932205]",574553.874247,True,Yes
0xad241d0,single-choice,"['>80%', '50%-80%', '30%-50%', '10%-30%', '<10%']","[134565.98764916285, 0, 0, 0.0, 0.0]",135221.887101,True,<10%
...,...,...,...,...,...,...
0x93cf3af,single-choice,"['sì', ' no']","[0.0, 653.7322378921115]",1141.553092,True,sì
0x4ebcc4c,single-choice,"['YES', 'NO']","[0.0, 472.81984655999344]",487.814446,True,YES


In [25]:
score_diffrences_dfs["Curve Finance"]["outcome_changed"].value_counts(normalize=True)

False    0.75
True     0.25
Name: outcome_changed, dtype: float64

One proposal which would've passed if not for whale intervention was this proposal to [add the XSTUSD-3CRV pair](https://snapshot.org/#/curve.eth/proposal/0x0eb23ea0b877666ad3ddcd0d7da0114acdfe5ae6390b5628b7509f4338022db5) to Curve's [gauge](https://resources.curve.fi/reward-gauges/understanding-gauges) [controller](https://curve.readthedocs.io/dao-gauges.html#the-gauge-controller) to accrue CRV for liquidity providers of XSTUSD-3CRV.

I don't know much about XSTUSD, apart from that it's a stable token deployed on Polkadot and Kusama that is backed by a synthetic token called XOR (Sora's native token). What was really interesting about this proposal though was the [governance discussion](https://gov.curve.fi/t/proposal-to-add-xstusd-3crv-to-the-gauge-controller/2998/15) on it.

![](./res/curve_governance_shenanigans.png)

What stood out to me (and was sorta a red flag) was XSTUSD's comparison with LUNA/UST. Now, this proposal was created before the whole [depegging disaster](https://rekt.news/luna-rekt/), but even before that, [quite a](https://twitter.com/runekek/status/1478166276979793922) [few people](https://twitter.com/FreddieRaynolds/status/1463960623402913797) had their concerns about it. So I checked out the first 16 accounts which showed really strong support for this proposal, and almost [every single](https://gov.curve.fi/u/meowtopia) [one was](https://gov.curve.fi/u/LiquidityKing) [created within](https://gov.curve.fi/u/Ryandotrrr) 2 days of the proposal's forum post. Clear signs of governance forum shenanigans that was poorly executed to raise hype for a proposal.

In [26]:
propsal_choices = plutocracy_report_data['Curve Finance'][plutocracy_report_data['Curve Finance']['proposal_id'] == '0x0eb23ea0b877666ad3ddcd0d7da0114acdfe5ae6390b5628b7509f4338022db5'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Curve Finance"].loc["0x0eb23ea"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Curve Finance'][plutocracy_report_data['Curve Finance']['proposal_id'] == '0x0eb23ea0b877666ad3ddcd0d7da0114acdfe5ae6390b5628b7509f4338022db5'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,Yes,No
Scores,45352.592228,529201.282019
Score Differeces,23462.994316,511330.472493


Just over 4% of voting power for this proposal was allocated by whales to vote "Yes" (which is just over half the total voting power allocated to the "Yes" choice for this proposal), whereas ~89% of whale voting power was allocated to voting "NO" (~97% of total voting power for this choice).

Anyway, I just wanted to highlight that whales are also sensible, and not always in the hands of the "evil" which I would classify as entities that would push proposals detrimental to the DAO for their own interests. It should also be noted that this was by far the proposal analyzed with the most voting power involved as well. It's also a good thing that CRV whales didn't have any incentive to pass this proposal, as well.

---
Moving away from DeFi and into developer infrastructure, let's look at Radicle

In [39]:
mask = score_diffrences_dfs["Radicle"].apply(
    lambda df: df["outcome_changed"] == True,
    axis=1
)

score_diffrences_dfs["Radicle"].loc[mask].astype({"total_vp": "float64"}, copy=False).sort_values("total_vp", ascending=False)

,type,choices,score_diffrences,total_vp,outcome_changed,outcome
Proposal ID,,,,,,
QmepPgXwo,single-choice,"['Aye', 'Nay']","[20708.27072226024, 90218.6509325973]",201262.680259,True,Aye
0x4c289dd,single-choice,"['Yes', 'No', 'Abstain']","[22191.20317652479, 0, 0.0]",54157.179860,True,No


We see that 18% of proposal (2 proposals) outcomes would've been changed had it not be for whale intervention.

In [28]:
score_diffrences_dfs["Radicle"]["outcome_changed"].value_counts(normalize=True)

False    0.818182
True     0.181818
Name: outcome_changed, dtype: float64

One such proposal was the one to [distribute some RAD leftover](https://snapshot.org/#/gov.radicle.eth/proposal/QmepPgXwo5q9GipZFKa32rnxaYoo3LrfRqduinftbU3L3S) from a Liquidity Bootsrapping (LBP) round conducted in February '21. This leftover RAD was proposed to be redistributed to participants of the LBP, i.e. people who bought RAD in this period from the Balancer LBP for RAD tokens.

In [52]:
propsal_choices = plutocracy_report_data['Radicle'][plutocracy_report_data['Radicle']['proposal_id'] == 'QmepPgXwo5q9GipZFKa32rnxaYoo3LrfRqduinftbU3L3S'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Radicle"].loc["QmepPgXwo"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Radicle'][plutocracy_report_data['Radicle']['proposal_id'] == 'QmepPgXwo5q9GipZFKa32rnxaYoo3LrfRqduinftbU3L3S'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,Aye,Nay
Scores,78136.278442,123126.401817
Score Differeces,20708.270722,90218.650933


Just over 10% of whale voting power for this proposal was allocated to voting "Aye" on this proposal (~27% of voting power allocated to the "Aye" choice came from whales). Whereas ~45% of voting power from whales was allocated to the "Nay" choice (~73% of whale voting power allocated to "Nay").

However, It's not obvious that whales were simply looking out for their own interests on this one. It's worth debating the [intentions of those who took part in these discussions](https://radicle.community/t/structured-discussion-distribute-lbp-unsold-tokens-to-the-people-who-contributed-to-the-lbp-in-proportion-to-their-contribution/1944/61) for this proposal for its passing. **Some context is always required when judging the actions of voters in DAO proposals**.

In [36]:
mask = score_diffrences_dfs["Gitcoin"].apply(
    lambda df: df["outcome_changed"] == True,
    axis=1
)

score_diffrences_dfs["Gitcoin"].loc[mask].astype({"total_vp": "float64"}, copy=False).sort_values("total_vp", ascending=False)

,type,choices,score_diffrences,total_vp,outcome_changed,outcome
Proposal ID,,,,,,
0xa94a8e1,single-choice,"['Yes', 'No', 'Abstain']","[4137843.9873177106, 5731585.734615851, 149817...",1.137574e+07,True,Yes
0xb154305,single-choice,"['OPTION 1 - $390,000', 'OPTION 2 - $330,000',...","[274316.18501807004, 8530118.546664951, 290753...",9.279563e+06,True,"OPTION 1 - $390,000"
0x68899d0,single-choice,"['FOR', 'AGAIINST', 'ABSTAIN']","[240657.94484809227, 7721191.48387823, 154011....",8.132303e+06,True,FOR
0xf8a7f4c,single-choice,"['FOR GR14 Struct & FOR New Elig', 'FOR GR14 S...","[675200.8295147448, 7344323.335575335, 1953.99...",8.027376e+06,True,FOR GR14 Struct & FOR New Elig
0x77b9cfe,single-choice,"['For', 'Against']","[2756719.14175241, 4222988.280091829]",6.990016e+06,True,For


In [37]:
mask = plutocracy_report_data_filtered["Gitcoin"].apply(
    lambda df: "0xa94a8e1" in df["proposal_id"],
    axis=1
)

plutocracy_report_data_filtered["Gitcoin"].loc[mask].iloc[0]["proposal_id"]

'0xa94a8e14396957870f827bc1f7526ba2b6d40f6e2a4c6fc7947e16c435f2d533'

In [31]:
score_diffrences_dfs["Gitcoin"]["outcome_changed"].value_counts(normalize=True)

False    0.9
True     0.1
Name: outcome_changed, dtype: float64